In [ ]:
using Revise
using BSON
using DataDeps, MAT, MLUtils

Traverse and preprocess all collected `mat` data as follows with proper thresholds.

In [ ]:
using NPZ
T = Float32
file = matopen("./data_sac_ns_new1.mat")
x_data = T.(collect(read(file, "a")))
y_data = T.(collect(read(file, "u")))
close(file)

file = matopen("./data_sac_ns_new2.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

file = matopen("./data_sac_ns_new3.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

@show size(x_data)
@show size(y_data)

target = npzread("./target.npz")
u_target = target["u"]
xu_target = ones(size(x_data))
yu_target = ones(size(y_data))

yu_target[1, :] .= u_target[:,end-1,11]
yu_target[:, :] .= yu_target[1:1, :]

threshold = 0.12
# threshold =  1.1
# pf: 1, not pf: 0
pf_labels = ones(size(x_data))
# safe: 1, not safe: 0
safe_labels = -ones(size(x_data))
not_pf = 0
pf_training = 0
for i in 1:size(x_data, 1)
    for j in 1:size(x_data, 2)
        if abs(y_data[i,j]-yu_target[i,j]) > threshold
            
            safe_labels[i,j] = 0
        end
    end
    for j in 1:size(x_data, 2)
        if safe_labels[i,end+1-j] == -1
            safe_labels[i,end+1-j] = 1
        else
            break
        end
    end
    if abs(y_data[i,end]-yu_target[i,end]) > threshold
        pf_labels[i, :] .= 0
        not_pf += 1
    else
        # pf and used for training
        pf_labels[i, :] .= 1
        pf_training += 1
    end
end

matwrite("data_sac_ns_new_abs_0.12.mat", Dict(
	"a" => x_data,
	"u" => y_data,
    "pf" => pf_labels,
    "safe" => safe_labels
))
@show not_pf, pf_training, 10000-not_pf-pf_training

In [ ]:
using NPZ
T = Float32
file = matopen("./data_opt_ns__0init_1.mat")
x_data = T.(collect(read(file, "a")))
y_data = T.(collect(read(file, "u")))
close(file)

file = matopen("./data_opt_ns__0init_2.mat")
    
x_data = cat(x_data, T.(collect(read(file, "a"))),dims=1)
y_data = cat(y_data, T.(collect(read(file, "u"))),dims=1)
close(file)

@show size(x_data)
@show size(y_data)

target = npzread("./target.npz")
u_target = target["u"]
xu_target = ones(size(x_data))
yu_target = ones(size(y_data))

yu_target[1, :] .= u_target[:,end-1,11]
yu_target[:, :] .= yu_target[1:1, :]

threshold = 0.145
# pf: 1, not pf: 0
pf_labels = ones(size(x_data))
# safe: 1, not safe: 0
safe_labels = -ones(size(x_data))
not_pf = 0
pf_training = 0
for i in 1:size(x_data, 1)
    for j in 1:size(x_data, 2)
        if abs(y_data[i,j]-yu_target[i,j]) > threshold
            
            safe_labels[i,j] = 0
        end
    end
    # @show  safe_labels[i,:]
    for j in 1:size(x_data, 2)
        if safe_labels[i,end+1-j] == -1
            safe_labels[i,end+1-j] = 1
        else
            break
        end
    end
    if abs(y_data[i,end]-yu_target[i,end]) > threshold
        pf_labels[i, :] .= 0
        not_pf += 1
    else
        # pf and used for training
        pf_labels[i, :] .= 1
        pf_training += 1
    end
end

matwrite("data_opt_ns_abs_0.145.mat", Dict(
	"a" => x_data,
	"u" => y_data,
    "pf" => pf_labels,
    "safe" => safe_labels
))
@show not_pf, pf_training, 10000-not_pf-pf_training